# 가설
- 아이템의 카테고리별로 월별 판매량이 다를것이다
- 가게별로 아이템의 평균 가격이 다를것이다.가격에 따라 판매량에 차이가 있을것이다.
- 피쳐의 수를 간추리면 결과예측에 긍정적인 영향을 미칠것이다.

# Import data

In [465]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from googletrans import Translator 

In [466]:
df_test = ("../files/test.csv")
df_train = ("../files/sales_train_v2.csv")

df_test = pd.read_csv(df_test, sep = ",") #(214200, 3)_
df_train = pd.read_csv(df_train, sep = ",") # (2935849, 6)
df_train['index'] = df_train.index
df_test['index'] = df_test.index

In [467]:
df_items = ("../files/items.csv")
df_item_categories = ("../files/item_categories.csv")
df_shops = ("../files/shops.csv")

df_items = pd.read_csv(df_items, sep = ",") #(214200, 3)
df_item_categories = pd.read_csv(df_item_categories, sep = ",") # (2935849, 6) 
df_shops = pd.read_csv(df_shops, sep = ",") #(214200, 3)

In [468]:
df_train.isnull().sum() #no null
df_test.isnull().sum() #no null

ID         0
shop_id    0
item_id    0
index      0
dtype: int64

In [469]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index
0,02.01.2013,0,59,22154,999.00,1.0,0
1,03.01.2013,0,25,2552,899.00,1.0,1
2,05.01.2013,0,25,2552,899.00,-1.0,2
3,06.01.2013,0,25,2554,1709.05,1.0,3
4,15.01.2013,0,25,2555,1099.00,1.0,4


In [470]:
df_test.head()

,ID,shop_id,item_id,index
0,0,5,5037,0
1,1,5,5320,1
2,2,5,5233,2
3,3,5,5232,3
4,4,5,5268,4


In [471]:
df_item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [472]:
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


# Translate

- 물건 카테고리이름, 상점이름등은 해석한 뒤 큰 키워드를 기준으로 전처리


- 아이템명의 경우 약 20000개정도의 값, 불필요한 정보라 생각해 드롭.


- 데이터프레임끼리 merge 하며 편집, 아이템명은 드롭하지 않음



In [473]:
translator = Translator()
item_category_name_ko = pd.Series([str(translator.translate(category_name, dest = "ko").text) for category_name in df_item_categories.item_category_name])
df_item_categories.item_category_name = item_category_name_ko
df_item_categories.item_category_name


shop_name_ko = pd.Series([str(translator.translate(shop_nm, dest = "ko").text) for shop_nm in df_shops.shop_name])
df_shops.shop_name = shop_name_ko
df_shops.shop_name

0                        Yakutsk Ordzhonikidze, 56 프랑
1                                     야쿠 츠크 TC "중앙"프랑
2                                  아디 게아 쇼핑 센터 "Mega"
3                              발라 시카 TRK "10 월 키노 미르"
4            볼츠키 (Volzhsky) 쇼핑 센터 "볼가 몰 (Volga Mall)"
5                       볼 로그 다 쇼핑 및 엔터테인먼트 센터 "마멀레이드"
6                          보로 네즈 (Porkhanovskaya, 13)
7                                Voronezh TRC "막시 미르"
8                            Voronezh TRC 도시 공원 "졸업생"
9                                               외환 거래
10                          Zhukovsky Str. 차 칼로프 39m?
11                       Zhukovsky Str. Chkalov 39 m²
12                                       온라인 상점 비상 사태
13                                    카잔 TC "Bahetle"
14                                   카잔 TC "파크 하우스"II
15                                   칼루가 TRC "XXI 세기"
16                                          콜로세움 "리오"
17    크라스 노야 르 스크 (Krasnoyarsk) 쇼핑 센터 "Vzletka Plaza"
18             크라스 노야 르 스크 (

# Simplify Categories 
- item_category_name의 카테고리를 줄여주려 가장 넓은 범위 선택

In [474]:
def split_words(df_col_values):
    tokenized = [i.split(" ") for i in df_col_values]
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 
    
    for i in range(len(df_col_values)):
        df_col_values[i] = tokenized[i][0] if hangul.sub('',tokenized[i][0]) else "기타"
    return df_col_values

df_item_category= pd.DataFrame({"item_category_id" : df_item_categories.item_category_name.index, "new_item_category_name": split_words(df_item_categories.item_category_name.values)})
df_item_category

,item_category_id,new_item_category_name
0,0,기타
1,1,액세서리
2,2,액세서리
3,3,액세서리
4,4,액세서리
5,5,액세서리
6,6,액세서리
7,7,액세서리
8,8,티켓
9,9,상품


# Add direction info

In [475]:
df_shops['direction'] = None
df_shops.loc[0:1,'direction'] = '동부'
df_shops.loc[20:33,'direction'] = '서부'
df_shops.loc[2,'direction'] = '서부'
df_shops.loc[3,'direction'] ='서부' 
df_shops.loc[4,'direction'] = '서부'
df_shops.loc[5,'direction'] = '서부'
df_shops.loc[6:8,'direction'] = '서부'
df_shops.loc[9,'direction'] = '기타'
df_shops.loc[10:11,'direction'] = '기타'
df_shops.loc[12,'direction'] = '기타'
df_shops.loc[13:14,'direction'] = '서부'
df_shops.loc[15,'direction'] = '서부'
df_shops.loc[16,'direction'] = '기타'
df_shops.loc[17:18,'direction'] = '중부'
df_shops.loc[34:35,'direction'] = '서부'
df_shops.loc[19,'direction'] = '서부'
df_shops.loc[36:37,'direction'] ='중부'
df_shops.loc[38,'direction']='중부'
df_shops.loc[39:41,'direction']='서부'
df_shops.loc[42:43,'direction']='서부'
df_shops.loc[44:45,'direction']='서부'
df_shops.loc[46,'direction']='서부'
df_shops.loc[47,'direction']='중부'
df_shops.loc[48,'direction']='중부'
df_shops.loc[49:51,'direction']='중부'
df_shops.loc[52:53,'direction']='서부'
df_shops.loc[54,'direction']='서부'
df_shops.loc[55:56,'direction']='기타'
df_shops.loc[57:58,'direction']= '동부'
df_shops.loc[59,'direction']='서부'
df_shops

,shop_name,shop_id,direction
0,"Yakutsk Ordzhonikidze, 56 프랑",0,동부
1,"야쿠 츠크 TC ""중앙""프랑",1,동부
2,"아디 게아 쇼핑 센터 ""Mega""",2,서부
3,"발라 시카 TRK ""10 월 키노 미르""",3,서부
4,"볼츠키 (Volzhsky) 쇼핑 센터 ""볼가 몰 (Volga Mall)""",4,서부
5,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",5,서부
6,"보로 네즈 (Porkhanovskaya, 13)",6,서부
7,"Voronezh TRC ""막시 미르""",7,서부
8,"Voronezh TRC 도시 공원 ""졸업생""",8,서부
9,외환 거래,9,기타


# Merge & Drop

In [476]:
#아이템카테고리 id만 df_train,df_test에 존재 -> items + item_categories -> df_train + item_info
df_item_info = pd.merge(df_item_category, df_items, on = "item_category_id")
df_item_info.head(5)

,item_category_id,new_item_category_name,item_name,item_id
0,0,기타,PC: Гарнитура HyperX Cloud Core игровая стерео...,5441
1,0,기타,Наушники PHILIPS SBC HC8680,16255
2,0,기타,Наушники RITMIX RH-120,16256
3,0,기타,Наушники RITMIX RH-124 Black,16257
4,1,액세서리,PS2: Карта памяти 8 Мб черная (Memory Card 8Mb...,5606


In [477]:
del df_item_info["item_name"] #item_name의 데이터는 너무 방대함 -> 카테고리로 충분하다 판단

In [478]:
df_item_info.head(5)

,item_category_id,new_item_category_name,item_id
0,0,기타,5441
1,0,기타,16255
2,0,기타,16256
3,0,기타,16257
4,1,액세서리,5606


In [479]:
df_train = pd.merge(df_train,df_item_info,on = "item_id")
df_test = pd.merge(df_test,df_item_info,on = "item_id")

In [480]:
del df_item_info
del df_item_categories
del df_items

In [481]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,new_item_category_name
0,02.01.2013,0,59,22154,999.0,1.0,0,37,시네마
1,23.01.2013,0,24,22154,999.0,1.0,3270,37,시네마
2,20.01.2013,0,27,22154,999.0,1.0,17081,37,시네마
3,02.01.2013,0,25,22154,999.0,1.0,25918,37,시네마
4,03.01.2013,0,25,22154,999.0,1.0,25919,37,시네마


In [482]:
df_test.head()

,ID,shop_id,item_id,index,item_category_id,new_item_category_name
0,0,5,5037,0,19,게임
1,5100,4,5037,5100,19,게임
2,10200,6,5037,10200,19,게임
3,15300,3,5037,15300,19,게임
4,20400,2,5037,20400,19,게임


In [483]:
df_train = pd.merge(df_train, df_shops, on = "shop_id")
df_test = pd.merge(df_test, df_shops, on = "shop_id")
del df_shops

In [484]:
df_test.head()

,ID,shop_id,item_id,index,item_category_id,new_item_category_name,shop_name,direction
0,0,5,5037,0,19,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",서부
1,1,5,5320,1,55,음악,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",서부
2,2,5,5233,2,19,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",서부
3,3,5,5232,3,23,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",서부
4,4,5,5268,4,20,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",서부


In [485]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,new_item_category_name,shop_name,direction
0,02.01.2013,0,59,22154,999.0,1.0,0,37,시네마,"야로 슬라 블 쇼핑 센터 ""Altair""",서부
1,16.05.2013,4,59,2573,249.0,1.0,498629,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",서부
2,26.01.2013,0,59,2574,399.0,1.0,77959,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",서부
3,09.01.2013,0,59,2574,399.0,1.0,77961,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",서부
4,24.02.2013,1,59,2574,399.0,1.0,133013,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",서부


# Direction To Shop Id

In [487]:
df_train["shop_id"] = df_train["direction"]
df_test["shop_id"] = df_test["direction"]
del df_test["direction"]
del df_train["direction"]

In [488]:
df_test

,ID,shop_id,item_id,index,item_category_id,new_item_category_name,shop_name
0,0,서부,5037,0,19,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
1,1,서부,5320,1,55,음악,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
2,2,서부,5233,2,19,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
3,3,서부,5232,3,23,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
4,4,서부,5268,4,20,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
5,5,서부,5039,5,23,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
6,6,서부,5041,6,20,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
7,7,서부,5046,7,55,음악,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
8,8,서부,5319,8,55,음악,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
9,9,서부,5003,9,20,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""


In [489]:
df_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,new_item_category_name,shop_name
0,02.01.2013,0,서부,22154,999.0,1.0,0,37,시네마,"야로 슬라 블 쇼핑 센터 ""Altair"""
1,16.05.2013,4,서부,2573,249.0,1.0,498629,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""
2,26.01.2013,0,서부,2574,399.0,1.0,77959,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""
3,09.01.2013,0,서부,2574,399.0,1.0,77961,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""
4,24.02.2013,1,서부,2574,399.0,1.0,133013,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""
5,29.03.2013,2,서부,2574,399.0,1.0,334238,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""
6,19.07.2013,6,서부,2574,399.0,1.0,706298,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""
7,03.07.2013,6,서부,2574,399.0,1.0,706299,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""
8,06.07.2013,6,서부,2574,399.0,1.0,706307,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""
9,28.08.2013,7,서부,2574,399.0,1.0,736350,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair"""


# Price Rank Per Shops and Items

- 가격정보를 명목형 값으로 해석, 최소화해야한다는 생각을 하였다.


- 한 아이템내의 가격을 sort 한 뒤 오름차순으로 등수를 매겼다.


- 그 후 한 아이템내에서 상대적인 가격 이라는 정보를 가진 컬럼이 생김

In [490]:
df_grouped = df_train.groupby(["shop_id","item_category_id","item_id"])["item_price"].apply(list)
#같은 제품에 있는 가격들을 list로 묶음
ranked_price_mat = []
for price_list in df_grouped.values:
    ranked_price_dict = {}
    ranked_price = {price : no  for no,price in enumerate(sorted(np.unique(price_list)))}
    ranked_price_dict.update(ranked_price)
    ranked_price_mat.append(ranked_price_dict)
    
    
# 가격에 따라 enumerate로 등수를 매김, 현재 가격과 등수가 각각 dict값의 키와 밸류로 저장되어있음

for value_no in range(len(df_grouped.values)): 
    for index in range(len(df_grouped.values[value_no])):
        df_grouped.values[value_no][index] = ranked_price_mat[value_no][df_grouped.values[value_no][index]]
del ranked_price_mat
del ranked_price_dict
    #df_grouped에 변경사항 반영

df_get_back = df_train.groupby(["shop_id","item_category_id","item_id"])["index"].apply(list)
get_back_dict = {}
for value_no in range(len(df_grouped.values)): 
    for index in range(len(df_grouped.values[value_no])):
        get_back_dict.update({df_get_back.values[value_no][index] : df_grouped.values[value_no][index]+1})
        
    #item_price를 원래대로 돌리기 위해 index값을 똑같이 묶은 뒤 딕트타입으로 키값에 인덱스번호를 부여
    #그리고 enumerate한 등수가 0이되지 않도록 1을 더해줌. {인덱스값 : 가격} 형태의 딕트 완성
del df_grouped       
df_ranked = pd.DataFrame(list(get_back_dict.items()),columns=['index','new_item_price'])
df_ranked = pd.merge(df_train,df_ranked, on = "index")

In [491]:
df_ranked

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,new_item_category_name,shop_name,new_item_price
0,02.01.2013,0,서부,22154,999.0,1.0,0,37,시네마,"야로 슬라 블 쇼핑 센터 ""Altair""",2
1,16.05.2013,4,서부,2573,249.0,1.0,498629,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",2
2,26.01.2013,0,서부,2574,399.0,1.0,77959,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",14
3,09.01.2013,0,서부,2574,399.0,1.0,77961,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",14
4,24.02.2013,1,서부,2574,399.0,1.0,133013,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",14
5,29.03.2013,2,서부,2574,399.0,1.0,334238,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",14
6,19.07.2013,6,서부,2574,399.0,1.0,706298,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",14
7,03.07.2013,6,서부,2574,399.0,1.0,706299,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",14
8,06.07.2013,6,서부,2574,399.0,1.0,706307,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",14
9,28.08.2013,7,서부,2574,399.0,1.0,736350,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",14


In [492]:
df_ranked.isnull().sum()

date                      0
date_block_num            0
shop_id                   0
item_id                   0
item_price                0
item_cnt_day              0
index                     0
item_category_id          0
new_item_category_name    0
shop_name                 0
new_item_price            0
dtype: int64

# Apply STD

- 아이템의 상대적인 가격정보컬럼으로 가격변동이 큰 가게인지, 가격변동이 작은 가게인지 유추

In [493]:
df_std_price_rank_shop = df_ranked_light.groupby(["shop_name"], as_index = True)["new_item_price"].std() #null 0
df_price_rank_per_shop = pd.DataFrame({"shop_name" : df_std_price_rank_shop.index, "std_price_rank_shop" : df_std_price_rank_shop.values})

df_std_price_rank_item = df_ranked_light.groupby(["new_item_category_name"], as_index = True)["new_item_price"].std().fillna(0) #null 2 mode 0
df_price_rank_per_item = pd.DataFrame({"new_item_category_name" :df_std_price_rank_item.index, "std_price_rank_item" : df_std_price_rank_item.values})

In [494]:
df_price_rank_per_item.head()

,new_item_category_name,std_price_rank_item
0,게임,22.385881
1,공식,0.600003
2,기타,23.587663
3,도서,1.877945
4,사무실,430.590140


In [495]:
df_price_rank_per_shop.head()

,shop_name,std_price_rank_shop
0,"Khimki ТЦ ""메가""",32.252748
1,"Mytischi TRK ""XL-3""",81.080647
2,"N.Novgorod TRC ""RIO""",98.453609
3,"RostovNaDon TRK ""Megacenter Horizon""",74.750293
4,"RostovNaDonu TC ""메가""",47.061304


In [496]:
df_train = pd.merge(df_price_rank_per_item, df_train, on = "new_item_category_name")
df_train = pd.merge(df_price_rank_per_shop, df_train, on = "shop_name")

In [497]:
df_test = pd.merge(df_price_rank_per_item, df_test, on = "new_item_category_name")
df_test= pd.merge(df_price_rank_per_shop, df_test, on = "shop_name")

In [498]:
del df_price_rank_per_item
del df_price_rank_per_shop
del df_train["shop_name"]
del df_test["shop_name"]

del df_test["index"]
del df_train["index"]

del df_train["item_category_id"]
del df_test["item_category_id"]

del df_train["item_price"]
del df_train["date"]

In [499]:
df_train.head()

,std_price_rank_shop,new_item_category_name,std_price_rank_item,date_block_num,shop_id,item_id,item_cnt_day
0,32.252748,게임,22.385881,0,서부,2705,1.0
1,32.252748,게임,22.385881,2,서부,2705,1.0
2,32.252748,게임,22.385881,2,서부,2705,1.0
3,32.252748,게임,22.385881,3,서부,2705,1.0
4,32.252748,게임,22.385881,4,서부,2705,1.0


In [500]:
df_train.isnull().sum()

std_price_rank_shop       0
new_item_category_name    0
std_price_rank_item       0
date_block_num            0
shop_id                   0
item_id                   0
item_cnt_day              0
dtype: int64

In [501]:
df_test.head()

,std_price_rank_shop,new_item_category_name,std_price_rank_item,ID,shop_id,item_id
0,98.453609,게임,22.385881,183600,서부,5037
1,98.453609,게임,22.385881,183602,서부,5233
2,98.453609,게임,22.385881,183603,서부,5232
3,98.453609,게임,22.385881,183604,서부,5268
4,98.453609,게임,22.385881,183605,서부,5039


In [502]:
df_test.isnull().sum()

std_price_rank_shop       0
new_item_category_name    0
std_price_rank_item       0
ID                        0
shop_id                   0
item_id                   0
dtype: int64

# Processing ,Grouping

In [503]:
df_test["date_block_num"] = 34 #2015 november
df_test.head()

,std_price_rank_shop,new_item_category_name,std_price_rank_item,ID,shop_id,item_id,date_block_num
0,98.453609,게임,22.385881,183600,서부,5037,34
1,98.453609,게임,22.385881,183602,서부,5233,34
2,98.453609,게임,22.385881,183603,서부,5232,34
3,98.453609,게임,22.385881,183604,서부,5268,34
4,98.453609,게임,22.385881,183605,서부,5039,34


In [504]:
df_train = df_train.groupby(["date_block_num","shop_id","new_item_category_name","std_price_rank_shop","std_price_rank_item","item_id"])["item_cnt_day"].mean().to_frame().reset_index()
df_train["item_cnt_month"] = None
df_train["item_cnt_month"] = df_train["item_cnt_day"]
del df_train["item_cnt_day"]

In [505]:
df_train.head()

,date_block_num,shop_id,new_item_category_name,std_price_rank_shop,std_price_rank_item,item_id,item_cnt_month
0,0,기타,게임,4.841655,22.385881,27,1.000000
1,0,기타,게임,4.841655,22.385881,1397,1.000000
2,0,기타,게임,4.841655,22.385881,1455,1.000000
3,0,기타,게임,4.841655,22.385881,1471,1.333333
4,0,기타,게임,4.841655,22.385881,1472,1.000000


In [506]:
df_test.head()

,std_price_rank_shop,new_item_category_name,std_price_rank_item,ID,shop_id,item_id,date_block_num
0,98.453609,게임,22.385881,183600,서부,5037,34
1,98.453609,게임,22.385881,183602,서부,5233,34
2,98.453609,게임,22.385881,183603,서부,5232,34
3,98.453609,게임,22.385881,183604,서부,5268,34
4,98.453609,게임,22.385881,183605,서부,5039,34


In [507]:
df_train.isnull().sum()

date_block_num            0
shop_id                   0
new_item_category_name    0
std_price_rank_shop       0
std_price_rank_item       0
item_id                   0
item_cnt_month            0
dtype: int64

In [508]:
df_test.isnull().sum()

std_price_rank_shop       0
new_item_category_name    0
std_price_rank_item       0
ID                        0
shop_id                   0
item_id                   0
date_block_num            0
dtype: int64

In [509]:
df_train.columns

Index(['date_block_num', 'shop_id', 'new_item_category_name',
       'std_price_rank_shop', 'std_price_rank_item', 'item_id',
       'item_cnt_month'],
      dtype='object')

In [510]:
df_test.columns

Index(['std_price_rank_shop', 'new_item_category_name', 'std_price_rank_item',
       'ID', 'shop_id', 'item_id', 'date_block_num'],
      dtype='object')

In [511]:
y_data = df_train.pop("item_cnt_month")
ID = df_test.pop("ID")
x_data = df_train
x_predict = df_test
len(x_predict),len(x_data)

(214200, 1609124)

In [512]:
df_all = pd.concat([x_data,x_predict])
df_all #null = 0

,date_block_num,item_id,new_item_category_name,shop_id,std_price_rank_item,std_price_rank_shop
0,0,27,게임,기타,22.385881,4.841655
1,0,1397,게임,기타,22.385881,4.841655
2,0,1455,게임,기타,22.385881,4.841655
3,0,1471,게임,기타,22.385881,4.841655
4,0,1472,게임,기타,22.385881,4.841655
5,0,1481,게임,기타,22.385881,4.841655
6,0,1523,게임,기타,22.385881,4.841655
7,0,1596,게임,기타,22.385881,4.841655
8,0,1833,게임,기타,22.385881,4.841655
9,0,1836,게임,기타,22.385881,4.841655


In [513]:
df_all = pd.get_dummies(df_all)
df_all.head()

,date_block_num,item_id,std_price_rank_item,std_price_rank_shop,new_item_category_name_게임,new_item_category_name_공식,new_item_category_name_기타,new_item_category_name_도서,new_item_category_name_사무실,new_item_category_name_상품,...,new_item_category_name_음악,new_item_category_name_지불,new_item_category_name_책,new_item_category_name_클린,new_item_category_name_티켓,new_item_category_name_프로그램,shop_id_기타,shop_id_동부,shop_id_서부,shop_id_중부
0,0,27,22.385881,4.841655,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1397,22.385881,4.841655,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,1455,22.385881,4.841655,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1471,22.385881,4.841655,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1472,22.385881,4.841655,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [514]:
x_data = df_all.iloc[:1609124,:]
x_predict = df_all.iloc[1609124:,:]
y_data = y_data.reshape(-1,1)
x_data.shape,y_data.shape

/home/hyuna/anaconda3/envs/deep_learning_study/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


((1609124, 25), (1609124, 1))

In [515]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_data = min_max_scaler.fit_transform(x_data)
x_predict = min_max_scaler.fit_transform(x_predict)
df_all = min_max_scaler.fit_transform(df_all)

In [516]:
df_all

array([[0.        , 0.00121792, 0.05066599, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06301592, 0.05066599, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06563219, 0.05066599, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.13541432, 0.03415162, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.1347377 , 0.03415162, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.55103974, 0.03415162, ..., 0.        , 0.        ,
        1.        ]])

# Predict

In [517]:
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

x_data = df_all[:1609124,:]
x_predict = df_all[1609124:,:]

x_train,x_test = x_data[0:round(len(x_data)*0.8)],x_data[round(len(x_data)*0.8):]
y_train,y_test = y_data[0:round(len(y_data)*0.8)],y_data[round(len(y_data)*0.8):] 

regr =linear_model.LinearRegression()
regr.fit(x_train, y_train)
y_pred = regr.predict(x_test)

print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [[ 3.52069443e-02  8.40933454e-03  2.73944687e+10 -8.70001597e-03
   4.48484719e+10  5.70011248e+09  4.47719069e+10  4.61550228e+10
   1.88419712e+10  3.93815976e+10  4.51754731e+10  4.54324292e+10
   4.56680864e+10  4.59917388e+10  4.60606526e+10  4.61089812e+10
   4.52329100e+10  4.61375875e+10  4.62052131e+10  4.61110346e+10
   4.53008745e+10  7.70123547e+11  7.70123547e+11  7.70123547e+11
   7.70123547e+11]]
Mean squared error: 1521546694800651264.00
Variance score: -1295014412295829504.00


In [520]:
regr.predict(x_predict)

array([[1.11828613],
       [1.11828613],
       [1.11828613],
       ...,
       [1.05908203],
       [1.05908203],
       [1.06262207]])

# Submission

In [528]:
y = pd.DataFrame(regr.predict(x_predict), index= ID,columns={"item_cnt_month":0})
y

,item_cnt_month
ID,
183600,1.118286
183602,1.118286
183603,1.118286
183604,1.118408
183605,1.118286
183606,1.118286
183609,1.118286
183611,1.118164
183613,1.118164


In [529]:
y.to_csv("submission_team2",sep = ",")